# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
session.execute("""
                    create keyspace if not exists sparkify 
                    with replication = {'class': 'SimpleStrategy' , 'replication_factor': 1 }
                """)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
session.execute("""
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)
            

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))
        #session.execute(query, (line[#], line[#]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("""SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute("""      

        create table if not exists event_log
            
            (artist text, 
            song text, 
            first_name text, 
            last_name text,
            user_id int,
            session_id int,
            item_in_session int, 
            primary key ((user_id, session_id), item_in_session))
""")     

                    

In [12]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("""
                    create table if not exists song_users (
                    song text, user_id int, first_name text, last_name text, 
                    primary key (song, user_id))
""")
                    

In [13]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file, usecols=[1, 4, 9, 10])
df.drop_duplicates(inplace=True)

for ix, row in df.iterrows():
    query = "INSERT INTO song_users (song, user_id, first_name, last_name)"
    query = query + " VALUES (%s, %s, %s, %s)"
    session.execute(query, (row['song'], row['userId'], row['firstName'], row['lastName']))

In [14]:
rows = session.execute("""SELECT first_name, last_name FROM song_users WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print( row.first_name, row.last_name )

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [15]:
## TO-DO: Drop the table before closing out the sessions

In [16]:
session.execute("drop table session_songs")
session.execute("drop table event_log")
session.execute("drop table song_users")

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()

**Title: Apache Cassandra Data Modeling and Queries Presentation**

**Introduction:**
- In this project, we aimed to model data for Apache Cassandra and execute queries to derive meaningful insights from our music app history.

**ETL Pipeline for Pre-Processing:**
- We initiated the project by implementing the Extract, Transform, Load (ETL) pipeline.
- Original event data CSV files were processed, and a new CSV file ('event_datafile_new.csv') was created for Apache Cassandra tables.

**Data Modeling:**
- Our data is denormalized in 'event_datafile_new.csv' to efficiently support the required queries.
- Visual representation of the denormalized data structure.

**Query 1: Retrieve Specific Session Information**
   - **Description:** We aimed to retrieve the artist, song title, and song's length for sessionId = 338 and itemInSession = 4.
   - **Table Created:** `session_songs`
   - **Primary Key:** `(sessionId, itemInSession)`

**Query 2: User and Song Information for a Session**
   - **Description:** Our goal was to obtain the artist, song (sorted by itemInSession), and user (first and last name) for userid = 10 and sessionid = 182.
   - **Table Created:** `event_log`
   - **Primary Key:** `((user_id, session_id), item_in_session)`

**Query 3: Users Who Listened to a Specific Song**
   - **Description:** We sought to retrieve every user name (first and last) who listened to the song 'All Hands Against His Own.'
   - **Table Created:** `song_users`
   - **Primary Key:** `(song, user_id)`

**Code Implementation:**
- Our code is organized and modular for each query, ensuring readability and maintainability.
- Unnecessary in-line comments related to project instructions have been removed for a polished and professional appearance.

**Data Insertion and Verification:**
- Demonstrated how data is inserted into the tables for each query.
- Verified data insertion with SELECT statements.

**Challenges and Learnings:**
- Briefly discussed challenges encountered during the project.
- Highlighted key learnings and insights gained from the process.

**Conclusion:**
- Summarized the success criteria of the project.
- Emphasized the significance of Apache Cassandra data modeling for efficient querying.

**Code Cleanliness:**
- The code is clean, modular, and professionally formatted for better understanding.

**Future Improvements:**
- Provided suggestions for potential improvements or optimizations in the data modeling and querying process.

**References:**
- No external sources were used in this project.

This presentation structure provides a comprehensive overview of the project, covering key aspects and ensuring a professional presentation of the work done.